In [1]:
import os
from pathlib import Path

In [2]:
os.getcwd()

'c:\\Users\\jhasu\\Desktop\\Mlops_template\\experiments'

In [3]:
os.chdir("../")

In [4]:
os.getcwd()

'c:\\Users\\jhasu\\Desktop\\Mlops_template'

In [11]:
from dataclasses import dataclass


@dataclass(frozen=True)
class modelevaluationentity:
    model_path: Path
    test_data_path: Path
    image_size: tuple
    batch_size: int

In [12]:
from project_name.Utills.utills import read_yaml
from project_name.contants import CONFIG_FILE_PATH, PARAMS_FILE_PATH


class configManager:
    def __init__(self,config = CONFIG_FILE_PATH,params = PARAMS_FILE_PATH):
        self.config =  read_yaml(config)
        self.params = read_yaml(params)
        os.makedirs(self.config.artifact_root,exist_ok=True)
    
    def get_evaluation_entity(self):
        config= self.config.model_evaluation
        entity = modelevaluationentity(
            model_path= config.model_dir,
            test_data_path= config.data_dir,
            image_size= self.params.image_size,
            batch_size= self.params.batch_size
        )
        return entity
    

In [13]:
from sklearn.metrics import accuracy_score, classification_report
import mlflow
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

class ModelEvaluation:
    def __init__(self, entity:modelevaluationentity):
        self.entity = entity
        self.device = "cuda" if torch.cuda.is_available() else "cpu"

    def evaluate(self):
        # Load model
        model = torch.load(self.entity.model_path, map_location=self.device)
        model.eval()

        
        transform = transforms.Compose([
            transforms.Resize(self.entity.image_size),
            transforms.ToTensor()
        ])
        test_dataset = datasets.ImageFolder(self.entity.test_data_path, transform=transform)
        test_loader = DataLoader(test_dataset, batch_size=self.entity.batch_size)

        y_true, y_pred = [], []

        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(self.device), labels.to(self.device)
                outputs = model(inputs)
                preds = torch.argmax(outputs, dim=1)

                y_true.extend(labels.cpu().numpy())
                y_pred.extend(preds.cpu().numpy())

        acc = accuracy_score(y_true, y_pred)
        report = classification_report(y_true, y_pred, target_names=test_dataset.classes, output_dict=True)

        # Log to MLflow
        mlflow.log_metric("accuracy", acc)
        mlflow.log_dict(report, "classification_report.json")